# Сервис по составлению меню питания на каждый день

Рабочее название `menu del dia`, `menu_del_dia_bot`

## Описание функционала
**Концепция**
* Это будет телеграм-бот (или email-рассылка), который высылает каждый вечер список блюд и их рецепты на следующий день, а возможно, и примерные расходы не продукты.
* Саначала нужно пройти опрос (в будущем после успешного опробования MVP):
    * на сколько дней в неделю выдать меню (1, 2,..., 7);
    * с какаого дня начинаем готовить (завтра, послезавтра, через неделю)
    * на сколько человек рассчитывать (или сколько порций);
    * едите или не едите мясо, грибы, мясо птицы, морепродукты (креветки, мидии), рыба белая, красная;
    * добавить фрукт на завтрак?
    * какие блюда в обеде: салат, суп, горячее (второе);
    * какие блюда в ужине: салат, суп, горячее (второе).
* В первом сообщении (перед началом 7 дневного периода) будет отправлено на ознакомление:
    * список блюда на каждый день с разделением на завтрак, обед и ужин;
    * список необходимых продуктов и их количество.
    * примерные затраты на питание на неделю (нужен предварительный ввод цен на продукты)
* Дисклеймер про аллергию и т.п.
* Куда писать отзывы и предложения: google-форма для конструктивной критики с заранее заданной структурой (или бот) или прям почту.

**Маинфест**
1. Я решаю свою проблему и мне неважно, что подобный софт уже есть. 
2. Хочу чтобы программа была такой, как я хочу. Хочу чтобы программа 
    * включала в себя только проверенные и опробованные мною блюда;
    * была в удобном для меня виде - телеграмм бот с минималистичным текстом и понятной последовательностью действий;
    * составляла список покупок на неделю вперёд.
    
**Идеи на будущее**
0. Нужно сделать генерировать список рецптов заранее и прописывать их в лог. При перезапуске сервера нужно начать с того же места и не отправлять заново то, что уже отправлено.
1. Возможность ставить лайки и дислайки. Куда-то сохранять эти реакции и ещё какую-нибудь добровольно полученную информацию о пользователях (пол, возраст, дети, регион)Потом давать чаще блюда с лайками.
2. Расчёт трат на продукты.
3. ML: рекомендации блюд. Это как в IVI: выбери 5...10 фильмов, и сделаем тебе рекомендации. Или сразу рекомендовать блюда, которые понравились таким же пользователям или ещё как-нибудь.
4. Время подготовки продуктов, время приготовления блюда. Можно поэкспериментировать и сделать модель, напрмиер `время = количество лукавиц * время резки одной лукавицы + количество каортофелин * время резки одной картофелины`  
5. Подсчёт калорий (это в самый хваост списка, т.к. мне это не нужно).
6. Возможность повторов блюд на неделе (True / False) или конкретное заданеи возможного количества повторов в виде числа.
7. Почитать https://journal.tinkoff.ru/list/chto-est/

## Типы блюд для MVP

Классификация по времени:
* Завтраки
* Обеды и ужины

Классификация по типу
* Салаты
* Супы
* Горячее

## Добыча рецептов

Всё вручную, т.к. нужно опробовать, переделать и переписать.

## База данных рецептов

### Создание

In [30]:
import pandas as pd
import sqlite3

In [4]:
path = '..\\db\\recipies.xlsx'

In [20]:
df_recipes = pd.read_excel(path, sheet_name='recipe')
df_recipes.columns

Index(['recipe_id', 'food_category_id', 'meal_type_id', 'recipe_name',
       'prep_time', 'cook_time', 'recipe_steps', 'link_to_original'],
      dtype='object')

In [9]:
df_food_category = pd.read_excel(path, sheet_name='food_category')
df_food_category.columns

Index(['food_category_id', 'food_category_name'], dtype='object')

In [10]:
df_meal_type = pd.read_excel(path, sheet_name='meal_type')
df_meal_type.columns

Index(['meal_type_id', 'meal_type_name'], dtype='object')

In [13]:
df_ingredients = pd.read_excel(path, sheet_name='ingredients')
df_ingredients.columns

Index(['ingredient_id', 'ingredient_name'], dtype='object')

In [17]:
df_recipe_ingredients = pd.read_excel(
    path, sheet_name='recipe_ingredients',
    names=['recipe_id', 'ingredient_id', 'ingredient_qty', 'measurement_id']
)
df_recipe_ingredients.columns

Index(['recipe_id', 'ingredient_id', 'ingredient_qty', 'measurement_id'], dtype='object')

In [31]:
# Create SQLite database connection
conn = sqlite3.connect('..\\db\\recipes.db')

# Write DataFrames to SQLite tables
df_recipes.to_sql('recipe', conn, index=False, if_exists='replace')
df_food_category.to_sql('food_category', conn, index=False, if_exists='replace')
df_meal_type.to_sql('meal_type', conn, index=False, if_exists='replace')
df_ingredients.to_sql('ingredients', conn, index=False, if_exists='replace')
df_recipe_ingredients.to_sql('recipe_ingredients', conn, index=False, if_exists='replace')
df_measurements.to_sql('measurements', conn, index=False, if_exists='replace')

# Close database connection
conn.close()

In [36]:
# Example query: Retrieve all recipes with their names and preparation times

# Create SQLite database connection
conn = sqlite3.connect('..\\db\\recipes.db')

# Create a cursor object to interact with the database
cursor = conn.cursor()
cursor.execute('SELECT recipe_id, recipe_name FROM recipe')

# Fetch all rows from the result set
rows = cursor.fetchall()

# Print the fetched rows
for i, row in enumerate(rows):
    print(i, row)

# Close cursor and database connection
cursor.close()
conn.close()

0 (1, 'Хлопья кукурузные с молоком')
1 (2, 'Гранола с йогуртом')
2 (3, 'Оладьи без яиц')
3 (4, 'Манная каша на молоке')
4 (5, 'Рисовая каша на молоке')
5 (6, 'Гречневая каша с молоком')
6 (7, 'Овсяная каша на молоке')
7 (8, 'Яйца в смятку')
8 (9, 'Омлет')
9 (10, 'Сырники жареные')
10 (11, 'Сырники печёные')
11 (12, 'Творожная запеканка')
12 (13, 'Пшённая каша на молоке')
13 (14, 'Блины со сметаной')
14 (15, 'Салат «Витаминный»')
15 (16, 'Салат «Греческий»')
16 (17, 'Салат «Винегрет»')
17 (18, 'Салат «Цезарь»')
18 (19, 'Салат «Оливье»')
19 (20, 'Салат «Мимоза»')
20 (21, 'Салат «Сельдь под шубой»')
21 (22, 'Салат свекольный с чесноком')
22 (23, 'Суп-пюре с шампиньонами и картофелем')
23 (24, 'Томатный суп с фрикадельками')
24 (25, 'Борщ с индейкой')
25 (26, 'Суп с фрикадельками из индейки и шпинатом')
26 (27, 'Суп с фрикадельками из индейки и фасолью')
27 (28, 'Суп с курицей, стручковой фасолью и грибами')
28 (29, 'Уха по-фински')
29 (30, 'Тыквенный крем-суп')
30 (31, 'Суп-пюре с броккол

### Работа с базой

In [34]:
con = sqlite3.connect('../db/recipes.db')
cur = con.cursor()

In [13]:
# сколько порций? 
n = 4
recipe_name = 'Томатный суп с фрикадельками'
for ing, mass, in zip(db[recipe_name]['ingridients'], db[recipe_name]['ingredient_quantity']):
    print(ing, '-', n * mass, 'г')

фарш из мяса индейки - 125.0 г
картофель - 125.0 г
подмидор - 87.5 г
болгарский перец - 50.0 г
томатная паста - 37.5 г
лук репчатый - 37.5 г
молотая паприка - 1.75 г
сушёный укроп, лук и петрушка - 1.75 г
соль - 0 г
перец горошком - 0 г
растительное масло - 0 г


## Запуск скрипта по распианию

https://www.geeksforgeeks.org/scheduling-python-scripts-on-linux/

To automate the script using CRON, you can modify the script to execute the desired functionality and then create a CRON job to run the script at specific intervals. Here's the modified version of the Python script that should be run using CRON:


### Setting Up CRON Job:

1. Open your terminal or SSH into your server.
2. Open the CRON table for editing using the command:
   ```
   crontab -e
   ```
3. Add a new CRON job to schedule the script to run every day at a specific time. For example, to run the script every day at 6 PM, add the following line to the CRON table:
   ```
   0 18 * * * /path/to/python3 /path/to/recipe_bot.py
   ```
   Replace `/path/to/python3` with the path to your Python 3 interpreter and `/path/to/recipe_bot.py` with the full path to your Python script.
4. Save and exit the editor.

Now, the CRON job will run the `recipe_bot.py` script every day at 6 PM. Make sure to adjust the CRON schedule and file paths according to your requirements and server setup.

In this script, sent recipes are logged in a file named recipe_bot.log along with timestamps. The get_last_sent_timestamp function reads the log file to retrieve the timestamp of the last sent recipe, allowing the script to resume sending recipes from the correct point.

Make sure to adjust the log file path, log format, and content according to your specific requirements. This approach simplifies the setup and provides a straightforward way to maintain a history of sent recipes.

In [ ]:
import os
import random
import logging
from datetime import datetime, timedelta
from telegram import Bot

# Configure logging
logging.basicConfig(filename='recipe_bot.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define SQLite queries
GET_WEEKLY_RECIPES = 'SELECT recipe_name FROM recipes ORDER BY RANDOM() LIMIT 7'
GET_DAILY_RECIPES = 'SELECT recipe_name FROM recipes ORDER BY RANDOM() LIMIT 3'

# Define Telegram bot token and chat ID
TELEGRAM_TOKEN = 'YOUR_TELEGRAM_BOT_TOKEN'
CHAT_ID = 'YOUR_CHAT_ID'

# Helper function to send messages
def send_message(recipes):
    bot = Bot(token=TELEGRAM_TOKEN)
    message = "\n".join(recipes)
    bot.send_message(chat_id=CHAT_ID, text=message)
    # Log sent recipes
    logging.info(f'Sent recipes: {recipes}')

# Get last sent recipe timestamp from the log file
def get_last_sent_timestamp():
    try:
        with open('recipe_bot.log', 'r') as log_file:
            lines = log_file.readlines()
            if lines:
                last_line = lines[-1]
                timestamp_str = last_line.split(' - ')[0]
                return datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S,%f')
    except FileNotFoundError:
        pass
    return None

# Get weekly recipes and send them
def send_weekly_recipes():
    recipes = cursor.execute(GET_WEEKLY_RECIPES).fetchall()
    recipes = [recipe[0] for recipe in recipes]
    send_message(recipes)

# Get daily recipes and send them
def send_daily_recipes():
    recipes = cursor.execute(GET_DAILY_RECIPES).fetchall()
    recipes = [recipe[0] for recipe in recipes]
    send_message(recipes)

# Check current day and send appropriate recipes
current_day = datetime.now().weekday()

last_sent_timestamp = get_last_sent_timestamp()

if current_day == 6:  # Sunday (Index 6 in Python's datetime)
    send_weekly_recipes()
else:
    if last_sent_timestamp is None or (datetime.now() - last_sent_timestamp).days > 1:
        send_daily_recipes()


## Восстановаление после аварийного завершения программы

Нужно подготовиться к выключению сервера.

`random_state` должно быть задано конкретным числом

Можно каждое 
* расписание на неделю
* каждое отправленное сообщение 
логировать и ввести счётчики отработавших циклов. 

После перезапуска програмы отпарвить сообщение и сбое на сервере и начать пролграмму с тех значений счётчика, на которых программа остановилась в прошлый раз - эти значения можно взять из логов.

А как быть когда много пользователей? В логах прописывать ещё и id пользователей

## Ссылки, которые помогли
* https://realpython.com/beautiful-soup-web-scraper-python/
* https://dbeaver.com/2022/06/23/import-data-with-dbeaver/
* https://stackoverflow.com/questions/38463361/creating-a-recipe-database
* https://stackoverflow.com/questions/54005899/how-do-i-properly-design-a-data-model-for-recipe-ingredient-using-mongodb
* https://infoeda.com/skolko-v-stolovoj-lozhke-gramm-i-millilitrov-tablica.html

## Затраченное время


## Чему научился:
1. проектирование базы данных;
2. использвоание SQL-запросов к базе данных в Python-коде;
3. использование DBeaver в качестве СУБД;
